In [1]:
from datasets import load_dataset
import pandas as pd
import json

# Load the dataset from Hugging Face
dataset = load_dataset('TIGER-Lab/MathInstruct')  # Replace 'dataset_name' with the name of the dataset you want to load

# Convert to a Pandas DataFrame
df = pd.DataFrame(dataset['train'])  # Use 'train', 'test', or 'validation' based on your needs
df

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Found cached dataset json (/Users/bryan/.cache/huggingface/datasets/TIGER-Lab___json/TIGER-Lab--MathInstruct-5c606c36c8364f7c/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

,instruction,output,source
0,The distance between two stars is 6.52 × 10^5 ...,Let's think about the multi-choice question.\n...,data/CoT/aqua_rat.json
1,How many ways can the letters in the word COMM...,Let's solve the multi-choice question step by ...,data/CoT/aqua_rat.json
2,A team of six entered for a shooting competiti...,"answers = ['A', 'B', 'C', 'D', 'E']\n# If the ...",data/PoT/aqua_rat_filtered.json
3,A psychiatrist has 4 patients that need 25 ses...,The second patient needs 6+5 = 11 sessions\n25...,data/CoT/gsm_rft.json
4,The radius of a wheel is 22.4 cm. What is the ...,radius = 22.4\nresolutions = 500\n# calculate ...,data/PoT/aqua_rat_filtered.json
...,...,...,...
262780,There are 81 people in a certain community who...,Let's think about the multi-choice question.\n...,data/CoT/aqua_rat.json
262781,How many 3 digit numbers can be made from the ...,Let's solve the multi-choice question step by ...,data/CoT/aqua_rat.json
262782,The sum of the present ages of a father and hi...,Let's think about the multi-choice question st...,data/CoT/aqua_rat.json
262783,A 2D image has a size of 256 x 256 pixels. App...,When you apply the Fourier transform to a 2D i...,data/CoT/math50k_camel.json


In [2]:
df = df[df['source'].str.contains('PoT')]
df

,instruction,output,source
2,A team of six entered for a shooting competiti...,"answers = ['A', 'B', 'C', 'D', 'E']\n# If the ...",data/PoT/aqua_rat_filtered.json
4,The radius of a wheel is 22.4 cm. What is the ...,radius = 22.4\nresolutions = 500\n# calculate ...,data/PoT/aqua_rat_filtered.json
6,"At a pool party, there are 4 pizzas cut into 1...",# define the initial number of slices\ntotal_s...,data/PoT/gsm_gpt4.json
7,Noel bakes 4 dozen donuts for his class. There...,# define the variables\ntotal_donuts = 4 * 12 ...,data/PoT/gsm_gpt4.json
9,louie takes out a 3 - month loan of $ 1000 . t...,n0 = 3.0\nn1 = 1000.0\nn2 = 10.0\nn3 = 3.0\nt0...,data/PoT/mathqa.json
...,...,...,...
262754,"Four of the six numbers 1867, 1993, 2019, 2025...","# define the list of numbers\nnumbers = [1867,...",data/PoT/MATH_train.json
262767,a vendor sells 60 percent of apples he had and...,n0 = 60.0\nn1 = 15.0\nn2 = 50.0\nt0 = n1 / 100...,data/PoT/mathqa.json
262771,"let p be a prime number . if n = 7 p , how man...",n0 = 7.0\nt0 = 1.0 / 2.0\nt1 = n0 / 2.0\nt2 = ...,data/PoT/mathqa.json
262777,Two partners (Moses and Norris) invested Rs 50...,total_profit = 5500\nmoses_investment = 5000\n...,data/PoT/aqua_rat_filtered.json


In [3]:
df['source'].value_counts()
#dont want numglue, aqua_rat, or mathqa because the questions are terribly written

data/PoT/mathqa.json               24382
data/PoT/gsm_gpt4.json             14591
data/PoT/numglue.json              13474
data/PoT/MATH_train.json           10632
data/PoT/aqua_rat_filtered.json     9772
data/PoT/crowdsourced.json           745
data/PoT/TheoremQA.json              703
Name: source, dtype: int64

In [4]:
df = df[df['source'].str.contains('TheoremQA')|df['source'].str.contains('MATH')|df['source'].str.contains('gpt')|df['source'].str.contains('crowdsourced')]

In [5]:
# Define a regular expression pattern to split text into sentences using . or ?
import re
sentence_pattern = r'([.?])'

def remove_program_sentences(text):
    # Split the text into sentences using the sentence_pattern regular expression.
    sentences = re.split(sentence_pattern, text)
    
    # Initialize variables to store sentences and punctuation
    filtered_sentences = []
    
    for sentence in sentences:
        if 'program' not in sentence.lower():
                filtered_sentences.append(sentence)
    
    # Join the remaining sentences back into a single string.
    sentences = ''.join(filtered_sentences)
    #Fix double punctuation problem
    if "?." in sentences:
        sentences = sentences.split("?.")[0]
        sentences = sentences + "?"
    if ".." in sentences:
        sentences = sentences.split("..")[0]
        sentences = sentences + "."
    return sentences
# Apply the function to the DataFrame column
df['instruction'] = df['instruction'].apply(remove_program_sentences)

<ipython-input-5-670380cebf10>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['instruction'] = df['instruction'].apply(remove_program_sentences)


In [6]:
pd.set_option('display.max_colwidth', -1)
df

<ipython-input-6-fdffb7a42c85>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,instruction,output,source
6,"At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left?",# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices),data/PoT/gsm_gpt4.json
7,"Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat?",# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student),data/PoT/gsm_gpt4.json
23,The planet Mercury travels around the Sun with a mean orbital radius of 5.8x10^10 m. The mass of the Sun is 1.99x10^30 kg. Use Newton's version of Kepler's third law to determine how long it takes Mercury to orbit the Sun. Give your answer in Earth days.,import math\ndef solve():\n G = 6.6743 * 10**(-11) # gravitational constant\n M = 1.99 * 10**30 # mass of the Sun\n r = 5.8 * 10**10 # mean orbital radius of Mercury\n T = 2 * math.pi * math.sqrt(r**3 / (G * M)) # Kepler's third law\n T = T / (60 * 60 * 24) # convert seconds to days\n return T\nprint(solve()),data/PoT/TheoremQA.json
32,Gus eats 2 eggs-omelet for breakfast. He has an egg salad sandwich made with 3 eggs for lunch. He then has an egg drop soup made with 1 egg for dinner. How many eggs did Gus eat altogether?,# define the number of eggs Gus eats for each meal\nbreakfast_eggs = 2\nlunch_eggs = 3\ndinner_eggs = 1\n\n# calculate the total number of eggs Gus eats\ntotal_eggs = breakfast_eggs + lunch_eggs + dinner_eggs\n\n# print the result\nprint(total_eggs),data/PoT/gsm_gpt4.json
35,"The function $f(x)$ satisfies\n\[f(x + y) = f(x) f(y)\]for all real numbers $x$ and $y.$ If $f(2) = 3,$ find $f(6).",def f(x):\n return 3**(x/2)\n\nprint(f(6)),data/PoT/MATH_train.json
...,...,...,...
262735,"Sara used $\frac{3}{10}$ of a roll of wrapping paper to wrap three presents. If she used the same amount of wrapping paper on each present, what fraction of a roll of wrapping paper did she use on each present?",# define the fraction of the roll used and the number of presents\ntotal_fraction = 3/10\nnum_presents = 3\n\n# calculate the fraction used per present\nfraction_per_present = total_fraction / num_presents\n\n# print the result\nprint(fraction_per_present),data/PoT/MATH_train.json
262741,"Diana wants to buy winter clothes for all the 40 children at her local children's home. The home has five times as many teenagers as toddlers. There are also some newborns. If there are 6 toddlers, for how many newborns will Diana be shopping?",# define the total number of children\ntotal_children = 40\n\n# define the number of toddlers\ntoddlers = 6\n\n# calculate the number of teenagers\nteenagers = 5 * toddlers\n\n# calculate the number of newborns\nnewborns = total_children - toddlers - teenagers\n\n# print the number of newborns\nprint(newborns),data/PoT/gsm_gpt4.json
262746,"Compute the integral $\iint_D xy^2 dA$, where $D$ is the rectangle defined by 0 <= x <= 2 and 0 <= y <= 1.","def solve():\n from sympy import symbols, integrate\n\n x, y = symbols('x y')\n f = x * y**2\n integral = integrate(integrate(f, (x, 0, 2)), (y, 0, 1))\n ans = integral.evalf()\n return ans\nprint(solve())",data/PoT/TheoremQA.json
262754,"Four of the six numbers 1867, 1993, 2019, 2025, 2109, and 2121 have a mean (average) of 2008. What is the mean (average) of the other two numbers?","# define the list of numbers\nnumbers = [1867, 1993, 2019, 2025, 2109, 2121]\n\n# calculate the total sum of all numbers\n

In [7]:
#Theorem, math and crowdsource are challenge datasets, GSM8k is grade school and cannot determine mathqa because it is a mix
question_type = []
for i in range(0, len(df)):
    if 'Theorem' in df.iloc[i]['source']:
        question_type.append('challenge')
    if 'MATH' in df.iloc[i]['source']:
        question_type.append('challenge')   
    if 'crowdsource' in df.iloc[i]['source']:
        question_type.append('challenge')
    if 'gpt' in df.iloc[i]['source']:
        question_type.append('grade school')
df['question_type']=question_type

<ipython-input-7-281acf659195>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question_type']=question_type


In [8]:
df['output'] = "Question: " + df['instruction'] + "\n" + "Solution:\n" + df['output']
colstodrop = ['instruction', 'source']
df = df.drop(colstodrop, axis=1)
df

<ipython-input-8-bff638104883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['output'] = "Question: " + df['instruction'] + "\n" + "Solution:\n" + df['output']


,output,question_type
6,"Question: At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left?\nSolution:\n# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices)",grade school
7,"Question: Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat?\nSolution:\n# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student)",grade school
23,Question: The planet Mercury travels around the Sun with a mean orbital radius of 5.8x10^10 m. The mass of the Sun is 1.99x10^30 kg. Use Newton's version of Kepler's third law to determine how long it takes Mercury to orbit the Sun. Give your answer in Earth days.\nSolution:\nimport math\ndef solve():\n G = 6.6743 * 10**(-11) # gravitational constant\n M = 1.99 * 10**30 # mass of the Sun\n r = 5.8 * 10**10 # mean orbital radius of Mercury\n T = 2 * math.pi * math.sqrt(r**3 / (G * M)) # Kepler's third law\n T = T / (60 * 60 * 24) # convert seconds to days\n return T\nprint(solve()),challenge
32,Question: Gus eats 2 eggs-omelet for breakfast. He has an egg salad sandwich made with 3 eggs for lunch. He then has an egg drop soup made with 1 egg for dinner. How many eggs did Gus eat altogether?\nSolution:\n# define the number of eggs Gus eats for each meal\nbreakfast_eggs = 2\nlunch_eggs = 3\ndinner_eggs = 1\n\n# calculate the total number of eggs Gus eats\ntotal_eggs = breakfast_eggs + lunch_eggs + dinner_eggs\n\n# print the result\nprint(total_eggs),grade school
35,"Question: The function $f(x)$ satisfies\n\[f(x + y) = f(x) f(y)\]for all real numbers $x$ and $y.$ If $f(2) = 3,$ find $f(6).\nSolution:\ndef f(x):\n return 3**(x/2)\n\nprint(f(6))",challenge
...,...,...
262735,"Question: Sara used $\frac{3}{10}$ of a roll of wrapping paper to wrap three presents. If she used the same amount of wrapping paper on each present, what fraction of a roll of wrapping paper did she use on each present?\nSolution:\n# define the fraction of the roll used and the number of presents\ntotal_fraction = 3/10\nnum_presents = 3\n\n# calculate the fraction used per present\nfraction_per_present = total_fraction / num_presents\n\n# print the result\nprint(fraction_per_present)",challenge
262741,"Question: Diana wants to buy winter clothes for all the 40 children at her local children's home. The home has five times as many teenagers as toddlers. There are also some newborns. If there are 6 toddlers, for how many newborns will Diana be shopping?\nSolution:\n# define the total number of children\ntotal_children = 40\n\n# define the number of toddlers\ntoddlers = 6\n\n# calculate the number of teenagers\nteenagers = 5 * toddlers\n\n# calculate the number of newborns\nnewborns = total_children - toddlers - teenagers\n\n# print the number of newborns\nprint(newborns)",grade school
262746,"Question: Compute the integral $\iint_D xy^2 dA$, where $D$ is the rectangle defined by 0 <= x <= 2 and 0 <= y <= 1.\nSolution:\ndef solve():\n from sympy import symbols, integrate\n\n x, y = symbols('x y')\n f = x * y**2\n integral = integrate(integrate(f, (x, 0, 2)), (y, 0, 1))\n ans = integral.evalf()\n return ans\nprint(solve())",challenge
262754,"Question: Four of the six numbers 1867, 1993, 2019, 2025, 2109, and 2121 have a mean (average) of 2008. What is the mean (average) of the other two numbers?\nSolution:\n# define the list of numbers\nnumbe

In [9]:
instruction = []
for i in range(0, len(df)):
    if "challenge" in df.iloc[i]['question_type']:
        instruction.append("Write a challenge math problem and Python code to solve the problem.")
    if "grade school" in df.iloc[i]['question_type']:
        instruction.append("Write a grade school math word problem and Python code to solve the word problem.")
df['instruction']=instruction
df['input']=""
colstodrop = ['question_type']
df = df.drop(colstodrop, axis=1)
df

,output,instruction,input
6,"Question: At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left?\nSolution:\n# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices)",Write a grade school math word problem and Python code to solve the word problem.,
7,"Question: Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat?\nSolution:\n# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student)",Write a grade school math word problem and Python code to solve the word problem.,
23,Question: The planet Mercury travels around the Sun with a mean orbital radius of 5.8x10^10 m. The mass of the Sun is 1.99x10^30 kg. Use Newton's version of Kepler's third law to determine how long it takes Mercury to orbit the Sun. Give your answer in Earth days.\nSolution:\nimport math\ndef solve():\n G = 6.6743 * 10**(-11) # gravitational constant\n M = 1.99 * 10**30 # mass of the Sun\n r = 5.8 * 10**10 # mean orbital radius of Mercury\n T = 2 * math.pi * math.sqrt(r**3 / (G * M)) # Kepler's third law\n T = T / (60 * 60 * 24) # convert seconds to days\n return T\nprint(solve()),Write a challenge math problem and Python code to solve the problem.,
32,Question: Gus eats 2 eggs-omelet for breakfast. He has an egg salad sandwich made with 3 eggs for lunch. He then has an egg drop soup made with 1 egg for dinner. How many eggs did Gus eat altogether?\nSolution:\n# define the number of eggs Gus eats for each meal\nbreakfast_eggs = 2\nlunch_eggs = 3\ndinner_eggs = 1\n\n# calculate the total number of eggs Gus eats\ntotal_eggs = breakfast_eggs + lunch_eggs + dinner_eggs\n\n# print the result\nprint(total_eggs),Write a grade school math word problem and Python code to solve the word problem.,
35,"Question: The function $f(x)$ satisfies\n\[f(x + y) = f(x) f(y)\]for all real numbers $x$ and $y.$ If $f(2) = 3,$ find $f(6).\nSolution:\ndef f(x):\n return 3**(x/2)\n\nprint(f(6))",Write a challenge math problem and Python code to solve the problem.,
...,...,...,...
262735,"Question: Sara used $\frac{3}{10}$ of a roll of wrapping paper to wrap three presents. If she used the same amount of wrapping paper on each present, what fraction of a roll of wrapping paper did she use on each present?\nSolution:\n# define the fraction of the roll used and the number of presents\ntotal_fraction = 3/10\nnum_presents = 3\n\n# calculate the fraction used per present\nfraction_per_present = total_fraction / num_presents\n\n# print the result\nprint(fraction_per_present)",Write a challenge math problem and Python code to solve the problem.,
262741,"Question: Diana wants to buy winter clothes for all the 40 children at her local children's home. The home has five times as many teenagers as toddlers. There are also some newborns. If there are 6 toddlers, for how many newborns will Diana be shopping?\nSolution:\n# define the total number of children\ntotal_children = 40\n\n# define the number of toddlers\ntoddlers = 6\n\n# calculate the number of teenagers\nteenagers = 5 * toddlers\n\n# calculate the number of newborns\nnewborns = total_children - toddlers - teenagers\n\n# print the number of newborns\nprint(newborns)",Write a grade school math word problem and Python code to solve the word problem.,
262746,"Question: Compute the integral $\iint_D xy^2 dA$, where $D$ is the rectangle defined by 0 

In [12]:
df

,output,instruction,input
6,"Question: At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left?\nSolution:\n# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices)",Write a grade school math word problem and Python code to solve the word problem.,
7,"Question: Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat?\nSolution:\n# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student)",Write a grade school math word problem and Python code to solve the word problem.,
23,Question: The planet Mercury travels around the Sun with a mean orbital radius of 5.8x10^10 m. The mass of the Sun is 1.99x10^30 kg. Use Newton's version of Kepler's third law to determine how long it takes Mercury to orbit the Sun. Give your answer in Earth days.\nSolution:\nimport math\ndef solve():\n G = 6.6743 * 10**(-11) # gravitational constant\n M = 1.99 * 10**30 # mass of the Sun\n r = 5.8 * 10**10 # mean orbital radius of Mercury\n T = 2 * math.pi * math.sqrt(r**3 / (G * M)) # Kepler's third law\n T = T / (60 * 60 * 24) # convert seconds to days\n return T\nprint(solve()),Write a challenge math problem and Python code to solve the problem.,
32,Question: Gus eats 2 eggs-omelet for breakfast. He has an egg salad sandwich made with 3 eggs for lunch. He then has an egg drop soup made with 1 egg for dinner. How many eggs did Gus eat altogether?\nSolution:\n# define the number of eggs Gus eats for each meal\nbreakfast_eggs = 2\nlunch_eggs = 3\ndinner_eggs = 1\n\n# calculate the total number of eggs Gus eats\ntotal_eggs = breakfast_eggs + lunch_eggs + dinner_eggs\n\n# print the result\nprint(total_eggs),Write a grade school math word problem and Python code to solve the word problem.,
35,"Question: The function $f(x)$ satisfies\n\[f(x + y) = f(x) f(y)\]for all real numbers $x$ and $y.$ If $f(2) = 3,$ find $f(6).\nSolution:\ndef f(x):\n return 3**(x/2)\n\nprint(f(6))",Write a challenge math problem and Python code to solve the problem.,
...,...,...,...
262735,"Question: Sara used $\frac{3}{10}$ of a roll of wrapping paper to wrap three presents. If she used the same amount of wrapping paper on each present, what fraction of a roll of wrapping paper did she use on each present?\nSolution:\n# define the fraction of the roll used and the number of presents\ntotal_fraction = 3/10\nnum_presents = 3\n\n# calculate the fraction used per present\nfraction_per_present = total_fraction / num_presents\n\n# print the result\nprint(fraction_per_present)",Write a challenge math problem and Python code to solve the problem.,
262741,"Question: Diana wants to buy winter clothes for all the 40 children at her local children's home. The home has five times as many teenagers as toddlers. There are also some newborns. If there are 6 toddlers, for how many newborns will Diana be shopping?\nSolution:\n# define the total number of children\ntotal_children = 40\n\n# define the number of toddlers\ntoddlers = 6\n\n# calculate the number of teenagers\nteenagers = 5 * toddlers\n\n# calculate the number of newborns\nnewborns = total_children - toddlers - teenagers\n\n# print the number of newborns\nprint(newborns)",Write a grade school math word problem and Python code to solve the word problem.,
262746,"Question: Compute the integral $\iint_D xy^2 dA$, where $D$ is the rectangle defined by 0 

In [10]:
# Save DataFrame to a JSON file
df.to_json('mammoth_train.json', orient='records', lines=True)  # Saves each row as a separate JSON object in the file

In [20]:
import json

file_path = "mammoth_train.json"
data = []

with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            obj = json.loads(line)
            data.append(obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON on line: {line}")
output_file = "mammoth_train.json"
with open(output_file, 'w') as json_file:
    json.dump(data, json_file)